# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234770191225                   -0.50    7.0
  2   -7.250182680038       -1.81       -1.41    1.0
  3   -7.251147843118       -3.02       -2.24    1.0
  4   -7.251315167076       -3.78       -2.28    3.0
  5   -7.251335897791       -4.68       -2.72    1.0
  6   -7.251335116301   +   -6.11       -2.88    2.0
  7   -7.251338367411       -5.49       -3.29    1.0
  8   -7.251338767130       -6.40       -3.80    1.0
  9   -7.251338794457       -7.56       -4.18    2.0
 10   -7.251338797218       -8.56       -4.45    2.0
 11   -7.251338798603       -8.86       -5.20    1.0
 12   -7.251338798699      -10.02       -5.59    3.0
 13   -7.251338798703      -11.40       -5.87    1.0
 14   -7.251338798704      -11.81       -6.44    2.0
 15   -7.251338798705      -12.76       -6.62    2.0
 16   -7.251338798705      -13.65       -6.98    1.0
 17   -7.251338798705      -13.88       -7.49 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03197951900508811
[ Info: Arnoldi iteration step 2: normres = 0.7124589362311384
[ Info: Arnoldi iteration step 3: normres = 0.8125301908346361
[ Info: Arnoldi iteration step 4: normres = 0.4105015362225375
[ Info: Arnoldi iteration step 5: normres = 0.34416851445427665
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.35e-02, 7.65e-02, 2.32e-01, 2.27e-01, 7.27e-02)
[ Info: Arnoldi iteration step 6: normres = 0.43698773802701696
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.51e-02, 1.05e-01, 4.08e-01, 9.76e-02, 3.50e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10685763332451928
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (9.03e-04, 2.30e-02, 3.68e-02, 5.01e-02, 7.59e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11016965616415021
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.37e